Simulator: Start Here
=====================

This script simulates `Imaging` of a strong lens where the sky background is not subtracted from the image and therefore
appears in the dataset.

It is used to demonstrate sky background modeling in
the `autolens_workspace/*/modeling/imaging/features/sky_background.py` example.

__Model__

This script simulates `Imaging` of a 'galaxy-scale' strong lens where:

 - The lens galaxy's light profile is a `Sersic`.
 - The lens galaxy's total mass distribution is an `Isothermal` and `ExternalShear`.
 - The source galaxy's light is a `Sersic`.

__Start Here Notebook__

If any code in this script is unclear, refer to the `simulators/start_here.ipynb` notebook.

In [ ]:
%matplotlib inline
from pyprojroot import here
workspace_path = str(here())
%cd $workspace_path
print(f"Working Directory has been set to `{workspace_path}`")

from os import path
import autolens as al
import autolens.plot as aplt

__Dataset Paths__

The path where the dataset will be output.

In [ ]:
dataset_type = "imaging"
dataset_name = "sky_background"

dataset_path = path.join("dataset", dataset_type, dataset_name)

__Simulate__

Simulate the image using a `Grid2D` with the `OverSamplingIterate` object.

In [ ]:
grid = al.Grid2D.uniform(
    shape_native=(100, 100),
    pixel_scales=0.1,
    over_sampling=al.OverSamplingIterate(
        fractional_accuracy=0.9999, sub_steps=[2, 4, 8, 16]
    ),
)

Simulate a simple Gaussian PSF for the image.

In [ ]:
psf = al.Kernel2D.from_gaussian(
    shape_native=(11, 11), sigma=0.1, pixel_scales=grid.pixel_scales
)

Create the simulator for the imaging data, which defines the exposure time, background sky, noise levels and psf.

The input `subtract_background_sky=False` ensures the background sky is not subtracted from the image, but is 
included in the data and used to estimate its noise-map.

The `background_sky_level` is also increased to 5.0 electrons per second, which is much higer than the value of 
0.1 electrons per second used in previous examples. This is to better illustrate the sky background in the image
and how it can be modeled.

In [ ]:
simulator = al.SimulatorImaging(
    exposure_time=300.0,
    psf=psf,
    background_sky_level=5.0,
    add_poisson_noise=True,
    subtract_background_sky=False,
)

__Ray Tracing__

Setup the lens galaxy's light, mass and source galaxy light for this simulated lens.

In [ ]:
lens_galaxy = al.Galaxy(
    redshift=0.5,
    bulge=al.lp.Sersic(
        centre=(0.0, 0.0),
        ell_comps=al.convert.ell_comps_from(axis_ratio=0.9, angle=45.0),
        intensity=2.0,
        effective_radius=0.6,
        sersic_index=3.0,
    ),
    mass=al.mp.Isothermal(
        centre=(0.0, 0.0),
        einstein_radius=1.6,
        ell_comps=al.convert.ell_comps_from(axis_ratio=0.9, angle=45.0),
    ),
    shear=al.mp.ExternalShear(gamma_1=0.05, gamma_2=0.05),
)

source_galaxy = al.Galaxy(
    redshift=1.0,
    bulge=al.lp.SersicCore(
        centre=(0.0, 0.0),
        ell_comps=al.convert.ell_comps_from(axis_ratio=0.8, angle=60.0),
        intensity=4.0,
        effective_radius=0.1,
        sersic_index=1.0,
    ),
)


Use these galaxies to setup a tracer, which will generate the image for the simulated `Imaging` dataset.

In [ ]:
tracer = al.Tracer(galaxies=[lens_galaxy, source_galaxy])

Pass the simulator a tracer, which creates the image which is simulated as an imaging dataset.

In [ ]:
dataset = simulator.via_tracer_from(tracer=tracer, grid=grid)

We now plot the simulated `Imaging` dataset before outputting it to fits.

Note how unlike the `Tracer` image above, the simulated `Imaging` dataset includes the blurring effects of the 
telescope's PSF and also has noise.

In [ ]:
dataset_plotter = aplt.ImagingPlotter(dataset=dataset)
dataset_plotter.subplot_dataset()

__Output__

Output the simulated dataset to the dataset path as .fits files.

In [ ]:
dataset.output_to_fits(
    data_path=path.join(dataset_path, "data.fits"),
    psf_path=path.join(dataset_path, "psf.fits"),
    noise_map_path=path.join(dataset_path, "noise_map.fits"),
    overwrite=True,
)

__Visualize__

Output a subplot of the simulated dataset, the image and the tracer's quantities to the dataset path as .png files.

For a faster run time, the tracer visualization uses the binned grid instead of the iterative grid.

In [ ]:
mat_plot = aplt.MatPlot2D(output=aplt.Output(path=dataset_path, format="png"))

dataset_plotter = aplt.ImagingPlotter(dataset=dataset, mat_plot_2d=mat_plot)
dataset_plotter.subplot_dataset()
dataset_plotter.figures_2d(data=True)

tracer_plotter = aplt.TracerPlotter(tracer=tracer, grid=grid, mat_plot_2d=mat_plot)
tracer_plotter.subplot_tracer()
tracer_plotter.subplot_galaxies_images()

__Tracer json__

Save the `Tracer` in the dataset folder as a .json file, ensuring the true light profiles, mass profiles and galaxies
are safely stored and available to check how the dataset was simulated in the future. 

This can be loaded via the method `tracer = al.from_json()`.

In [ ]:
al.output_to_json(
    obj=tracer,
    file_path=path.join(dataset_path, "tracer.json"),
)

The dataset can be viewed in the folder `autolens_workspace/imaging/sky_background`.